# Szacowanie potrzeb wypożyczalni sprzętu narciarskiego

### Standardowe pakiety wykorzystywane w języky Python i algorytmów uczenia maszynowego

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Zastosujemy narzędzia z pakietu *revoscalepy*

In [2]:
from revoscalepy import RxComputeContext, RxInSqlServer, RxSqlServerData
from revoscalepy import rx_import


Express Edition will continue to be enforced.


In [3]:
conn_str = 'Driver=SQL Server;Server=.;Database=TutorialDB;Trusted_Connection=True;'

### Opis kolumn, które będą importowane do analiz

In [4]:
column_info = {
         "Year" : { "type" : "integer" },
         "Month" : { "type" : "integer" },
         "Day" : { "type" : "integer" },
         "RentalCount" : { "type" : "integer" },
         "WeekDay" : {
             "type" : "factor",
             "levels" : ["1", "2", "3", "4", "5", "6", "7"]
         },
         "Holiday" : {
             "type" : "factor",
             "levels" : ["1", "0"]
         },
         "Snow" : {
             "type" : "factor",
             "levels" : ["1", "0"]
         }
     }

### Pobierz dane z tabeli serwera SQL

In [5]:
data_source = RxSqlServerData(table="dbo.rental_data",
                               connection_string=conn_str, column_info=column_info)

#### Określ parametry z jakimi zostanie utworzony kontekst działania analiz

In [6]:
computeContext = RxInSqlServer(
     connection_string = conn_str,
     num_tasks = 1,
     auto_cleanup = False
)

#### Stwórz kontekst wewnątrz serwera SQL

In [7]:
RxInSqlServer(connection_string=conn_str, num_tasks=1, auto_cleanup=False)

#### Zaimportuj dane do obiektu dataframe z pakietu Pandas

In [13]:
df = pd.DataFrame(rx_import(input_data = data_source))
#print("Obiekt Data Frame:", df.stack())

Rows Read: 453, Total Rows Processed: 453, Total Chunk Time: 2.370 seconds 


In [14]:
df.head(n=6)

,Year,Month,Day,RentalCount,WeekDay,Holiday,Snow,FHoliday,FSnow,FWeekDay
0,2014,1,20,445,2,1,0,1,0,2
1,2014,2,13,40,5,0,0,0,0,5
2,2013,3,10,456,1,0,0,0,0,1
3,2014,3,31,38,2,0,0,0,0,2
4,2014,4,24,23,5,0,0,0,0,5
5,2015,2,11,42,4,0,0,0,0,4


#### Pobierz wszystkie kolumny z obiektu *df*

In [9]:
columns = df.columns.tolist()

#### Filtruj kolumny i usuń te, których nie chcesz używać do dalszej analizy

In [15]:
columns = [c for c in columns if c not in ["Year"]]

In [16]:
columns

['Month',
 'Day',
 'RentalCount',
 'WeekDay',
 'Holiday',
 'Snow',
 'FHoliday',
 'FSnow',
 'FWeekDay']

### Tworzenie modelu

#### Jaka zmienna, której predykcja wartości jest celem tworzonego modelu?

In [17]:
target = "RentalCount"

#### Stwórz zbiór uczący - argument *random_state* jest ziarnem dla generatora liczb losowych

In [18]:
train = df.sample(frac=0.8, random_state=1)

#### Zbiór testowy jest rałą reszą nie będącą zbiorem uczącym

In [19]:
test = df.loc[~df.index.isin(train.index)]

#### Pokaż rozmiar obydwu zbiorów

In [21]:
print("Rozmiar zbioru uczącego:", train.shape)
print("Rozmiar zbioru testowego:", test.shape)

Rozmiar zbioru uczącego: (362, 10)
Rozmiar zbioru testowego: (91, 10)


#### Stwórz instancję obiektu klasy *LinearRegression* pochodzącej z pakietu *sklearn.linear_model*

In [22]:
lin_model = LinearRegression()

#### Dopasuj model do danych ze zbioru uczącego

In [23]:
lin_model.fit(train[columns], train[target])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

#### Generuj dane prognozowane z wykorzystaniem zbudowanego modelu na danych pochodzących ze zbioru testowego

In [25]:
lin_predictions = lin_model.predict(test[columns])
print("Dane prognozowane:", lin_predictions)

Dane prognozowane: [  40.   38.  240.   39.  514.   48.  297.   25.  507.   24.   30.   54.
   40.   26.   30.   34.   42.  390.  336.   37.   22.   35.   55.  350.
  252.  370.  499.   48.   37.  494.   46.   25.  312.  390.   35.   35.
  421.   39.  176.   21.   33.  452.   34.   28.   37.  260.   49.  577.
  312.   24.   24.  390.   34.   64.   26.   32.   33.  358.  348.   25.
   35.   48.   39.   44.   58.   24.  350.  651.   38.  468.   26.   42.
  310.  709.  155.   26.  648.  617.   26.  846.  729.   44.  432.   25.
   39.   28.  325.   46.   36.   50.   63.]


#### Zweryfikuj w jakim stopniu model poprawnie prognozuje dane. Jako parametr określający poprawność działania modelu, użyjemy *średniego błędu kwadratowego*. 

In [28]:
lin_mse = mean_squared_error(lin_predictions, test[target])
print("Błąd obliczeń:", lin_mse)

Błąd obliczeń: 4.29528694732e-27


### Dalszy ciąg w Microsoft SQL Server Management Studio